In [19]:
import ultralytics
ultralytics.checks()


Ultralytics YOLOv8.2.32  Python-3.12.4 torch-2.3.1+cpu CPU (AMD Ryzen 5 5600G with Radeon Graphics)
Setup complete  (12 CPUs, 31.3 GB RAM, 106.1/930.9 GB disk)


In [20]:
#TODO Pascal VOC Format to YOLO Format
def xml_to_yolo_bbox(bbox, w, h):
    # bbox : xmin, ymin, xmax, ymax
    x_center = ((bbox[0]+bbox[2])/2)/w
    y_center = ((bbox[1]+bbox[3])/2)/h
    width = (bbox[2]-bbox[0])/w
    height = (bbox[3]-bbox[1])/h
    return [x_center, y_center, width, height]

In [21]:
import xml.etree.ElementTree as ET
import glob
import os
import json
from tqdm import tqdm

road_sign_root = 'C:/project/object_detection_load/road_sign'
annot_path = os.path.join(road_sign_root,"annotations")
img_path = os.path.join(road_sign_root, "image")
label_path = os.path.join(road_sign_root, "labels")

if not os.path.exists(label_path):
    os.makedirs(label_path)

In [22]:
# TODO pascal VOC Format to YOLO Format
# *.xml -> *txt
classes = []

# 1. xml 파일 리스트 불러오기
files = glob.glob(os.path.join(annot_path, '*.xml'))
print(files[0])

for fil in tqdm(files):
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]

    result = []

    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find('size').find('width').text)
    height = int(root.find('size').find('height').text)

    for obj in root.findall('object'):
        label = obj.find('name').text
        if label not in classes:
            classes.append(label)
        idx = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        bbox_string = "".join([str(x) for x in yolo_bbox])
        result.append(f"{idx} {bbox_string}")

    if result:
        with open(os.path.join(label_path, f"{filename}.txt"), "w", encoding = "utf") as f:
            f.write("|n".join(result))




IndexError: list index out of range

In [23]:
cd c:/project/object_detection_road/road_sign

c:\project\object_detection_road\road_sign


In [24]:
import os
import random
from shutil import copyfile, rmtree

road_sign_path = './road_sign_detection'
label_ext = '.txt'
img_ext = '.png'

# 저장 경로 만들기

folder_list = [
    'road_sign_detection/train', 
    'road_sign_detection/val', 
    'road_sign_detection/test',
    'road_sign_detection/', 
    'road_sign_detection/train/labels',
    'road_sign_detection/val/images', 
    'road_sign_detection/val/labels']

for folder in folder_list:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [25]:
# Todo 데이터셋 Train/Test 분리

# 1. file  목록 작성
file_list = os.listdir(os.path.join(road_sign_path, "images"))

# 2. shuffle
random.shuffle(file_list)

# 3. Train/Test 분리
test_ratio = 0.1
test_list = file_list[:int(len(file_list)*test_ratio)]
train_list = file_list[int(len(file_list)*test_ratio):]

print(f"train : {len(train_list)}, test : {len(test_list)}")

# 4. 파일 이동
for i in test_list:
    f_name = os.path.splitext(i)[0]
    copyfile(os.path.join(road_sign_path, 'images', f_name + img_ext), os.path.join(road_sign_path, 'val/images', f_name + img_ext))
    copyfile(os.path.join(road_sign_path, 'labels', f_name + label_ext), os.path.join(road_sign_path, 'val/labels', f_name + label_ext))

for i in train_list:
    f_name = os.path.splitext(i)[0]
    copyfile(os.path.join(road_sign_path, 'images', f_name + img_ext), os.path.join(road_sign_path, 'train/images', f_name + img_ext))
    copyfile(os.path.join(road_sign_path, 'labels', f_name + label_ext), os.path.join(road_sign_path, 'train/labels', f_name + label_ext))

train : 790, test : 87


In [26]:
import yaml
import os
from ultralytics import YOLO

# 데이터 설정
data = dict()
data['train'] = 'C:/project/object_detection_road/road_sign/road_sign_detection/train'
data['val'] = 'C:/project/object_detection_road/road_sign/road_sign_detection/val'
data['test'] = 'C:/project/object_detection_road/road_sign/road_sign_detection/test'

data['nc'] = 4
data['name'] = ['Traffic_light', 'Speedlimit', 'Crosswalk', 'Stop']

# config 파일 작성
config_path = 'C:/project/object_detection_road/train_model/road_sign.yaml'
os.makedirs(os.path.dirname(config_path), exist_ok=True)
with open(config_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)


In [12]:
cd C:/project/object_detection_road/Train_model/

C:\project\object_detection_load\Train_model


In [27]:

# Training
model = YOLO('yolov8s.yaml')

# 'data' 인자에 실제 config 파일 경로를 지정합니다.
results = model.train(data=config_path, 
                      epochs=50, 
                      batch=32, 
                      device='cpu', 
                      patience=30, 
                      name='road_sign_s')

Ultralytics YOLOv8.2.32  Python-3.12.4 torch-2.3.1+cpu CPU (AMD Ryzen 5 5600G with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8s.yaml, data=C:/project/object_detection_road/train_model/road_sign.yaml, epochs=50, time=None, patience=30, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=road_sign_s22, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sa

train: Scanning C:\project\object_detection_road\road_sign\road_sign_detection\train\labels... 877 images, 0 backgrounds, 877 corrupt: 100%|██████████| 877/877 [00:00<00:00, 880.05it/s]

train: WARNING  C:\project\object_detection_road\road_sign\road_sign_detection\train\images\road0.png: ignoring corrupt image/label: could not convert string to float: '0.57303370786516850.36750.411985018726591770.425'
train: WARNING  C:\project\object_detection_road\road_sign\road_sign_detection\train\images\road1.png: ignoring corrupt image/label: could not convert string to float: '0.5150.6077738515901060.260.7703180212014135'
train: WARNING  C:\project\object_detection_road\road_sign\road_sign_detection\train\images\road10.png: ignoring corrupt image/label: could not convert string to float: '0.43750.498127340823970030.3450.9737827715355806'
train: WARNING  C:\project\object_detection_road\road_sign\road_sign_detection\train\images\road100.png: ignoring corrupt image/label: could not convert string to float: '0.49750.429870129870129850.820.8337662337662337'
train: WARNING  C:\project\object_detection_road\road_sign\road_sign_detection\train\images\road101.png: ignoring corrupt imag

train: New cache created: C:\project\object_detection_road\road_sign\road_sign_detection\train\labels.cache
WARNING  No images found in C:\project\object_detection_road\road_sign\road_sign_detection\train\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


ValueError: not enough values to unpack (expected 3, got 0)

In [28]:
# todo validation
# 1. import library
from ultralytics import YOLO

# 2. road model
model = YOLO('best_road_sign.pt')

# 3. Validate the model
metrics = model.val()

# 4. validation 결과 print
print("map50-95", metrics.box.map)
print("map50", metrics.box.map50)

Ultralytics YOLOv8.2.32  Python-3.12.4 torch-2.3.1+cpu CPU (AMD Ryzen 5 5600G with Radeon Graphics)
YOLOv8s summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs


FileNotFoundError: 'road_sign.yaml' does not exist

In [29]:
cd c:/project/object_detection_road/road_sign


c:\project\object_detection_road\road_sign


C:\Users\user\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
from ultralytics import YOLO
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

class Annotator:
    def __init__(self, image, line_width=None, font_size=None):
        self.image = image
        self.lw = line_width or max(round(sum(image.shape) / 2 * 0.003), 2)
        self.fs = font_size or max(self.lw - 1, 1)
        self.colors = [(0, 255, 0), (255, 255, 0), (0, 0, 255), (255, 0, 0)]

    def box_label(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(self.image, (x1, y1), (x2, y2), color, thickness=self.lw, lineType=cv2.LINE_AA)
        if label:
            tf = max(self.lw - 1, 1)  # font thickness
            w, h = cv2.getTextSize(label, 0, fontScale=self.fs / 3, thickness=tf)[0]  # text width, height
            outside = y1 - h >= 3
            cv2.rectangle(self.image,
                          (x1, y1 - h if outside else y1),
                          (x1 + w, y1 + 3 if outside else y1 + h + 3),
                          color, -1, cv2.LINE_AA)  # filled
            cv2.putText(self.image,
                        label,
                        (x1, y1 - 2 if outside else y1 + h + 2),
                        0,
                        self.fs / 3,
                        txt_color,
                        thickness=tf,
                        lineType=cv2.LINE_AA)

    def result(self):
        return self.image

# 모델 로드
model = YOLO('best_road_sign.pt')

# 테스트 이미지 및 결과 디렉토리 설정
root_folder = 'C:/project/object_detection_road/road_sign/road_sign_detection/test/images'
result_folder = 'C:/project/object_detection_road/road_sign/road_sign_detection/result'
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

test_img_list = os.listdir(root_folder)

device = 'cpu'
color_dict = [(0, 255, 0), (255, 255, 0), (0, 0, 255), (255, 0, 0)]
color_dict2 = [(0, 0, 0), (0, 0, 0), (255, 255, 255), (255, 255, 255)]

# 이미지에 대해 추론 수행
for img_name in test_img_list:
    img_path = os.path.join(root_folder, img_name)
    img = cv2.imread(img_path)

    results = model.predict(source=img, device=device, save=False)

    for result in results:
        annotator = Annotator(img)
        for box in result.boxes:
            cls = int(box.cls[0])
            conf = box.conf[0]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f'{model.names[cls]} {conf:.2f}'
            annotator.box_label([x1, y1, x2, y2], label, color=color_dict[cls])

        annotated_img = annotator.result()

        # 결과 저장
        result_img_path = os.path.join(result_folder, img_name)
        cv2.imwrite(result_img_path, annotated_img)

        # 결과 출력
        plt.imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
